In [16]:
from EmbeddingModels.QwenEmbedder import QwenEmbedder
from EmbeddingModels.ModernBertEmbedder import ModernBertEmbedder
import chromadb
import uuid

In [5]:
chroma_client = chromadb.PersistentClient(path="./chroma_db")

In [10]:
qwen_embedder = QwenEmbedder("Qwen/Qwen3-Embedding-8B", use_fp16=True)

# 2. Ingest Documents (No instruction needed for docs usually)
docs = ["Deep learning is a subset of ML.", "Qwen is a model from Alibaba."]
doc_vectors_qwen = qwen_embedder.encode(docs, batch_size=2)

Loading Qwen/Qwen3-Embedding-8B on cuda...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
modern_embedder = ModernBertEmbedder(
    model_name="Alibaba-NLP/gte-modernbert-base",
    normalize=True
)
docs = [
    "Deep learning is a subset of ML.", "Qwen is a model from Alibaba."
]
doc_vectors_bert = modern_embedder.encode(docs, batch_size=2)


Loading Alibaba-NLP/gte-modernbert-base on cuda...


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [17]:
collection = chroma_client.get_or_create_collection(
    name="test_collection",
    metadata={"hnsw:space": "ip"}
)

In [18]:
text = "This is a test academic paper regarding transformer efficiency."
vector = modern_embedder.encode(text)[0]

In [20]:
collection.add(
    ids=["test_id_001"],          # Hardcoded ID
    embeddings=[vector.tolist()], # CRITICAL: Convert Numpy -> List
    documents=[text],             # Store the raw text
    metadatas=[{"type": "test"}]  # Optional tag
)
print(f"Collection Count: {collection.count()}") # Should be 1

Collection Count: 1


In [21]:
results = collection.query(
    query_embeddings=[vector.tolist()], # Search with the EXACT same vector
    n_results=1
)

print(f"Distance: {results['distances'][0][0]}")

Distance: 1.1920928955078125e-07
